In [118]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline 
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.linear_model import LinearRegression 
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.model_selection import KFold, GridSearchCV
os.chdir("E:/Training/AV/Big Mart III")

In [5]:
train = pd.read_csv("train_v9rqX0R.csv")
test = pd.read_csv("test_AbJTz2l.csv")
print(train.columns)

Index(['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Visibility',
       'Item_Type', 'Item_MRP', 'Outlet_Identifier',
       'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type', 'Item_Outlet_Sales'],
      dtype='object')


In [62]:
train.shape, test.shape

((8523, 12), (5681, 11))

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
items_trn = train[['Item_Identifier', 'Item_Weight']]
items_trn = items_trn.drop_duplicates()
items_tst = test[['Item_Identifier', 'Item_Weight']]
items_tst = items_tst.drop_duplicates()
items = pd.concat([items_trn, items_tst])
items = items.dropna()
items = items.drop_duplicates()
items.columns = ['Item_Identifier', 'I_Weight']
items.isnull().sum()

In [43]:
train_1 = train.merge(items, how='left', on='Item_Identifier')
train_1 = train_1.drop('Item_Weight', axis=1)
train_1.isnull().sum()

Item_Identifier                 0
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
I_Weight                        0
dtype: int64

In [45]:
train_1.columns

Index(['Item_Identifier', 'Item_Fat_Content', 'Item_Visibility', 'Item_Type',
       'Item_MRP', 'Outlet_Identifier', 'Outlet_Establishment_Year',
       'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type',
       'Item_Outlet_Sales', 'I_Weight'],
      dtype='object')

In [ ]:
outlets = train_1[['Outlet_Identifier', 'Outlet_Establishment_Year',
       'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type']]
outlets = outlets.drop_duplicates()
outlets

In [ ]:
outlets = outlets[['Outlet_Identifier','Outlet_Size']]
#outlets.to_csv("Outlets.csv", index=False)

In [69]:
outlets = outlets[['Outlet_Identifier','Outlet_Size']]
outlets = outlets.dropna()
outlets_imp = pd.DataFrame({'Outlet_Identifier':['OUT010','OUT017','OUT045'],
                            'Outlet_Size':["Small","High","High"]})
outlets = pd.concat([outlets,outlets_imp], ignore_index=True)
outlets.columns=['Outlet_Identifier','O_Size']
train_2 = train_1.merge(outlets, how='left', on='Outlet_Identifier')
train_2 = train_2.drop('Outlet_Size', axis=1)
train_2.shape


(8523, 12)

In [75]:
pd.crosstab( train_2['Outlet_Type'] , train_2['O_Size'] , margins=True)

O_Size,High,Medium,Small,All
Outlet_Type,,,,
Grocery Store,0,0,1083,1083
Supermarket Type1,2787,930,1860,5577
Supermarket Type2,0,928,0,928
Supermarket Type3,0,935,0,935
All,2787,2793,2943,8523


In [66]:
train_2.isnull().sum()

Item_Identifier                 0
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
I_Weight                        0
O_Size                          0
dtype: int64

OR

In [ ]:
outlets = train_1[['Outlet_Identifier','Outlet_Size']]
outlets = outlets.drop_duplicates()
size_map = {
    "OUT010": "Small",
    "OUT017": "High",
    "OUT045": "High"
}

train_1["Outlet_Size"] = train_1["Outlet_Size"].fillna(
    train_1["Outlet_Identifier"].map(size_map)
)
# Validating the processing
pd.crosstab( train_1['Outlet_Type'] , train_1['Outlet_Size'] , margins=True)

Outlet_Size,High,Medium,Small,All
Outlet_Type,,,,
Grocery Store,0,0,1083,1083
Supermarket Type1,2787,930,1860,5577
Supermarket Type2,0,928,0,928
Supermarket Type3,0,935,0,935
All,2787,2793,2943,8523


In [84]:
train_1['Item_Fat_Content'].value_counts()

Item_Fat_Content
Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: count, dtype: int64

In [87]:
train_1['Item_Fat_Content'].replace({'reg':'Regular',
                                   'LF':'Low Fat',
                                   'low fat':'Low Fat'},
                                  inplace=True)
train_1['Item_Fat_Content'].value_counts()

Item_Fat_Content
Low Fat    5517
Regular    3006
Name: count, dtype: int64

In [90]:
train_1.columns

Index(['Item_Identifier', 'Item_Fat_Content', 'Item_Visibility', 'Item_Type',
       'Item_MRP', 'Outlet_Identifier', 'Outlet_Establishment_Year',
       'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type',
       'Item_Outlet_Sales', 'I_Weight'],
      dtype='object')

In [ ]:
train_1['Item_Type'].value_counts()

In [93]:
train_1['Outlet_Type'].value_counts()

Outlet_Type
Supermarket Type1    5577
Grocery Store        1083
Supermarket Type3     935
Supermarket Type2     928
Name: count, dtype: int64

In [94]:
train_1

,Item_Identifier,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,I_Weight
0,FDA15,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,9.300
1,DRC01,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,5.920
2,FDN15,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,17.500
3,FDX07,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Small,Tier 3,Grocery Store,732.3800,19.200
4,NCD19,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,8.930
...,...,...,...,...,...,...,...,...,...,...,...,...
8518,FDF22,Low Fat,0.056783,Snack Foods,214.5218,OUT013,1987,High,Tier 3,Supermarket Type1,2778.3834,6.865
8519,FDS36,Regular,0.046982,Baking Goods,108.1570,OUT045,2002,High,Tier 2,Supermarket Type1,549.2850,8.380
8520,NCJ29,Low Fat,0.035186,Health and Hygiene,85.1224,OUT035,2004,Small,Tier 2,Supermarket Type1,1193.1136,10.600
8521,FDN46,Regular,0.145221,Snack Foods,103.1332,OUT018,2009,Medium,Tier 3,Supermarket Type2,1845.5976,7.210


0

In [ ]:
def clean_data(df):
    df_1 = df.merge(items, how='left', on='Item_Identifier')
    df_1 = df_1.drop('Item_Weight', axis=1)

    outlets = df_1[['Outlet_Identifier','Outlet_Size']]
    outlets = outlets.drop_duplicates()
    size_map = {
        "OUT010": "Small",
        "OUT017": "High",
        "OUT045": "High"
    }

    df_1["Outlet_Size"] = df_1["Outlet_Size"].fillna(
        df_1["Outlet_Identifier"].map(size_map)
    )

    df_1['Item_Fat_Content'] = df_1['Item_Fat_Content'].replace({'reg':'Regular',
                                   'LF':'Low Fat',
                                   'low fat':'Low Fat'})
    return df_1

train_1 = clean_data(train)
test_1 = clean_data(test)
# Validating the processing
#pd.crosstab( train_1['Outlet_Type'] , train_1['Outlet_Size'] , margins=True)
pd.crosstab( test_1['Outlet_Type'] , test_1['Outlet_Size'] , margins=True)

Outlet_Size,High,Medium,Small,All
Outlet_Type,,,,
Grocery Store,0,0,722,722
Supermarket Type1,621,1856,1240,3717
Supermarket Type2,0,618,0,618
Supermarket Type3,0,624,0,624
All,621,3098,1962,5681


In [134]:
train_1.shape, test_1.shape

((8523, 12), (5681, 11))

In [135]:
train.shape, test.shape

((8523, 12), (5681, 11))

In [136]:
test_1.isnull().sum()

Item_Identifier              0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
I_Weight                     0
dtype: int64

In [137]:
train_1.columns

Index(['Item_Identifier', 'Item_Fat_Content', 'Item_Visibility', 'Item_Type',
       'Item_MRP', 'Outlet_Identifier', 'Outlet_Establishment_Year',
       'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type',
       'Item_Outlet_Sales', 'I_Weight'],
      dtype='object')

In [138]:
X_train = train_1.drop(['Item_Identifier',
                                 'Outlet_Identifier','Item_Outlet_Sales'], axis=1)
X_test = test_1.drop(['Item_Identifier', 'Outlet_Identifier'], axis=1)
y_train = train_1['Item_Outlet_Sales']
ohe = OneHotEncoder()
col_trnf = ColumnTransformer([('OHE',ohe, make_column_selector(dtype_include=object) )],
                             remainder='passthrough', verbose_feature_names_out=False)
rf = RandomForestRegressor(random_state=25)
pipe = Pipeline([('TRNS', col_trnf),('MODEL',rf)])
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

In [139]:
ss = pd.read_csv("sample_submission_8RXa3c6.csv")
ss['Item_Outlet_Sales'] = y_pred 
ss.to_csv("ss_rf_20_dec_25.csv", index=False)

In [140]:
test.shape, ss.shape

((5681, 11), (5681, 3))

In [141]:
from lightgbm import LGBMRegressor

In [ ]:
l_gbm = LGBMRegressor(random_state=25, verbose=-1)
pipe = Pipeline([('TRNS', col_trnf),('MODEL',l_gbm)])
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

In [146]:
ss = pd.read_csv("sample_submission_8RXa3c6.csv")
ss['Item_Outlet_Sales'] = y_pred 
ss.to_csv("ss_lgb_20_dec_25.csv", index=False)